In [1]:
from sklearn.cluster import MeanShift
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('checkins_01.txt', sep='|', header = 0).replace(r'^\s+$', np.nan, regex = True).dropna()


C:\Users\gruyuri\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<bound method DataFrame.head of             id       user_id    venue_id        latitude        \
1           984222    15824.0      5222.0          38.8951118    
3           984234    44652.0      5222.0           33.800745    
7           984291   105054.0      5222.0          45.5234515    
9           984318  2146539.0      5222.0           40.764462    
10          984232    93870.0    380645.0          33.4483771    
11          984483  1030290.0    955969.0          32.2217429    
12          984685   304253.0     23558.0               40.65    
13          984470   720850.0    749715.0          33.4483771    
15          984610  1639666.0    442605.0           33.414768    
18          984653  1647192.0     23558.0          42.3584308    
19          984251   298547.0     77014.0          33.4483771    
20          984528  2046311.0     15682.0           33.414768    
25          984438  2096701.0      4432.0          33.4483771    
26          984277  1648816.0     18006.0   

In [3]:
data.head(5)

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
7,984291,105054.0,5222.0,45.5234515,-122.6762071,2012-04-21 17:39:22
9,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870.0,380645.0,33.4483771,-112.0740373,2012-04-21 17:38:18


In [7]:
X_ds = data[:100000]
X_ds.shape

(100000, 6)

In [16]:
#X_ds.columns = [u'id', u'user_id', u'venue_id', u'latitude', u'longitude', u'created_at']
X = np.array((X_ds['latitude'],X_ds['longitude']))
X = X.T

In [17]:
ms = MeanShift(bandwidth = 0.1, bin_seeding = True, n_jobs = -1)
ms.fit(X)

MeanShift(bandwidth=0.1, bin_seeding=True, cluster_all=True, min_bin_freq=1,
     n_jobs=-1, seeds=None)

In [18]:
labels = ms.labels_
cluster_centers = ms.cluster_centers_
label_unique = np.unique(labels)
n_clusters = len(label_unique)
print n_clusters

3091


In [22]:
members1 = labels == 0
t_arr = X[members1, 0]
print t_arr.size
print cluster_centers[0][0]
print cluster_centers[0][1]

12511
40.7177163973
-73.9918354199


In [25]:
df_centers = pd.DataFrame()

for k in range(n_clusters):
    members = labels == k
    t_arr = X[members, 0]
    if (t_arr.size > 15):
        cluster_center = cluster_centers[k]
        x0 = cluster_center[0]
        y0 = cluster_center[1]
        row = pd.Series([x0, y0],["X","Y"])
        df_centers = df_centers.append([row],ignore_index=True)

In [26]:
print df_centers

             X           Y
0    40.717716  -73.991835
1    33.446380 -111.901888
2    33.448410 -112.074004
3    41.878244  -87.629843
4    37.688682 -122.409330
5    38.886165  -77.048783
6    33.357345 -111.822654
7    33.766636  -84.393289
8    42.363219  -71.073688
9    47.606245 -122.332044
10   36.117229 -115.171073
11   34.060398 -118.248709
12   44.977948  -93.267301
13   30.267184  -97.743119
14   39.735830 -104.986580
15   40.767431  -73.819191
16   39.951680  -75.162736
17   34.035487 -118.438998
18   32.980893 -117.078118
19   32.803021  -96.769897
20   28.543502  -81.376429
21   32.711344 -117.153639
22   32.221713 -110.926535
23   29.762698  -95.382314
24   34.125047 -118.342722
25   25.785812  -80.217938
26   33.811233 -117.890612
27   43.040698  -87.907508
28   45.523483 -122.676280
29   37.408394 -122.106207
..         ...         ...
559  44.058173 -121.315310
560  32.319940 -106.763654
561  42.115474  -79.286368
562  42.026834  -71.204463
563  42.513971  -92.414068
5

In [30]:
offices_coords = np.array([[33.751277, -118.188740], 
                           [25.867736, -80.324116],
                           [51.503016, -0.075479],
                           [52.378894, 4.885084],
                           [39.366487, 117.036146],
                           [-33.868457, 151.205134]])
print offices_coords

[[  3.37512770e+01  -1.18188740e+02]
 [  2.58677360e+01  -8.03241160e+01]
 [  5.15030160e+01  -7.54790000e-02]
 [  5.23788940e+01   4.88508400e+00]
 [  3.93664870e+01   1.17036146e+02]
 [ -3.38684570e+01   1.51205134e+02]]


In [32]:
df_offices = pd.DataFrame(offices_coords)
df_offices.columns = [u'X', u'Y']
print df_offices

           X           Y
0  33.751277 -118.188740
1  25.867736  -80.324116
2  51.503016   -0.075479
3  52.378894    4.885084
4  39.366487  117.036146
5 -33.868457  151.205134


In [42]:
print df_offices.get_value(0,'X')
print df_offices.get_value(0,'Y')

33.751277
-118.18874


In [28]:
df_distance = df_centers
df_distance['X_off'] = 0.0
df_distance['Y_off'] = 0.0
df_distance['distance'] = 0.0

print df_distance.head(5)


           X           Y  X_off  Y_off  distance
0  40.717716  -73.991835    0.0    0.0       0.0
1  33.446380 -111.901888    0.0    0.0       0.0
2  33.448410 -112.074004    0.0    0.0       0.0
3  41.878244  -87.629843    0.0    0.0       0.0
4  37.688682 -122.409330    0.0    0.0       0.0


In [37]:
from math import sqrt
def euclid_distance(x1, y1, x2, y2):
    d = (x1-x2)**2 + (y1-y2)**2
    return sqrt(d)

print euclid_distance(0., 0., 1., 1.)

1.41421356237


In [43]:
for (idx, row) in df_centers.iterrows():
    x0 = row.loc['X']
    y0 = row.loc['Y']
    min_d = euclid_distance(x0, y0, df_offices.get_value(0,'X'), df_offices.get_value(0,'Y'))
    for j in range(5):
        dd = euclid_distance(x0, y0, df_offices.get_value(j+1,'X'), df_offices.get_value(j+1,'Y'))
        if (dd < min_d):
            min_d = dd
    df_distance.set_value(idx, 'distance', min_d)

print df_distance.head(5)

           X           Y  X_off  Y_off   distance
0  40.717716  -73.991835    0.0    0.0  16.143720
1  33.446380 -111.901888    0.0    0.0   6.294241
2  33.448410 -112.074004    0.0    0.0   6.122232
3  41.878244  -87.629843    0.0    0.0  17.598580
4  37.688682 -122.409330    0.0    0.0   5.772048


In [45]:
df_sort = df_distance.sort_values(['distance'])
print df_sort.head(20)

             X           Y  X_off  Y_off  distance
405 -33.860630  151.204776    0.0    0.0  0.007835
371  52.372964    4.892317    0.0    0.0  0.009353
299  25.883050  -80.307826    0.0    0.0  0.022358
58   51.502991   -0.125537    0.0    0.0  0.050058
51   33.809878 -118.148924    0.0    0.0  0.070848
25   25.785812  -80.217938    0.0    0.0  0.134109
167  25.705350  -80.283429    0.0    0.0  0.167406
191  33.931836 -118.141898    0.0    0.0  0.186536
91   26.010098  -80.199991    0.0    0.0  0.188876
85   33.888325 -118.048928    0.0    0.0  0.195779
42   33.872986 -118.362091    0.0    0.0  0.211811
538  26.116200  -80.393791    0.0    0.0  0.258049
47   33.643205 -117.946115    0.0    0.0  0.265606
55   26.120863  -80.158907    0.0    0.0  0.302270
26   33.811233 -117.890612    0.0    0.0  0.304098
11   34.060398 -118.248709    0.0    0.0  0.314884
161  26.200585  -80.250716    0.0    0.0  0.340846
33   33.731744 -117.825700    0.0    0.0  0.363565
17   34.035487 -118.438998    0